In [1]:
from utils.file_handling import *
from utils.misc import *
from utils.matplot_handlers import *
from utils.visualizers import *
from segmentation.composite_action_extractor import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from itertools import product
%matplotlib inline

# Entropy Parameters
# csv_file_name = 'Cartpole'
# csv_file_name = 'Mountaincar'
# csv_file_name = 'Pendulum'
csv_file_name = 'Acrobot'
episodes = None#[2]
analysis_method = ['w', 's', 'p', 'js', 'tv', 'kld']

# Read in entropy csv
base_path = get_absolute_path('data', directory_file_hint=csv_file_name.lower(), ignore_files=False)
csv_path = os.path.join(base_path, 'state_action_data.csv')

pre_data = pd.read_csv(csv_path, index_col=None)
selected_episodes = pre_data['episode'].unique()  # Either do [some index] or [:]
selected_episodes = [selected_episodes] if np.isscalar(selected_episodes) else selected_episodes
data_df = pre_data[pre_data['episode'].isin(selected_episodes)]

In [2]:
from ipywidgets import FloatSlider, IntSlider
from ipywidgets import interact, interactive, fixed, interact_manual
from copy import copy

matplotlib.rcParams["figure.facecolor"] = 'white'

def matplot_gen(i, do_return=False):
    fig = plt.figure(figsize=(9, 5))
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.subplot(1,2,1)
    plt.imshow(video[i])
    plt.subplot(1,2,2)
    plt.subplots_adjust(right=1.5)
    plt.plot(info['index_groups'][0], markevery=[i], marker='|', markeredgecolor='r', 
         markersize=40, mew=4)
    plt.plot(scored_composite_actions[0][0][0])
    plt.ylabel('Normalized / Windowed / \nThreshed / Measured Values')
    plt.xlabel('Time Steps')
    plt.title(f'Highest Effective Correlated Action Joint 0 Measurements for \n{csv_file_name} using {method}')
    if do_return: 
        im = get_img_from_fig(fig)
        plt.close()
        return im

In [3]:
import cv2
import imageio
# define a function which returns an image as numpy array from figure
def get_img_from_fig(fig, dpi=180):
    buf = io.BytesIO()
    fig.savefig(buf, format="png", bbox_inches="tight")
    buf.seek(0)
    img_arr = np.frombuffer(buf.getvalue(), dtype=np.uint8)
    buf.close()
    img = cv2.imdecode(img_arr, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img[::3, ::3, :]

In [4]:
selected_episodes

array([  4,  23,  34,  41,  43,  49,  67,  68, 134, 251, 419, 450])

In [ ]:
if episodes is None: episodes = selected_episodes
for episode, method in list(product(episodes, analysis_method)):
    print(f'Working on episode {episode}, method {method}')

    print(f'Loading Video')
    with open(os.path.join(base_path, 'video.pkl'), 'rb') as f:
        video_path = pickle.load(f)
    
    video = video_path[episode][0]
    
    print(f'Loading df')
    main_df = data_df[data_df['episode'] == episode].copy()
    print(f'Running Composite Action Extractor')
    composite_actions, info = CompositeActionExtractor.get_composite_actions(dataframe=main_df, 
                                                                             analysis_method=method, 
                                                                             window_size=5)
    
    ## We can take a look at primarily the action with the maximum total entropy
    scored_composite_actions = {key: [(seg, info['state_analyzed_norm'][~np.isnan(seg)].max()) for seg in composite_actions[key]] for key in composite_actions}
    k = 1
    for key in scored_composite_actions:
        scored_composite_actions[key] = list(sorted(scored_composite_actions[key], key=lambda x: x[1], reverse=True))[:k]

    isolated_composite_actions = {key: [seg[~np.isnan(seg)] for seg in composite_actions[key]] for key in composite_actions}

    print(f'Processing Images')
    images = [matplot_gen(i, True) for i in range((video.shape[0]-1))]
    print(f'Generating Gif')
    imageio.mimsave(f'../../video_samples/{csv_file_name.lower()}_episode_{episode}_{method}.gif', images)

Working on episode 4, method w
Loading Video
Loading df
Running Composite Action Extractor
Processing Images
Generating Gif
Working on episode 4, method s
Loading Video
Loading df
Running Composite Action Extractor
Processing Images
Generating Gif
Working on episode 4, method p
Loading Video
Loading df
Running Composite Action Extractor
Processing Images
Generating Gif
Working on episode 4, method js
Loading Video
Loading df
Running Composite Action Extractor
Processing Images
Generating Gif
Working on episode 4, method tv
Loading Video
Loading df
Running Composite Action Extractor
Processing Images


In [ ]:
import pickle

name = os.path.join(base_path, 'segments')
if name[-4:] != ".pkl":
    name += ".pkl"
with open(name, 'wb') as f:
    pickle.dump(isolated_composite_actions, f, pickle.HIGHEST_PROTOCOL)